In [1]:
import pandas as pd
import numpy as np

import json
import os

from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_json('/Users/Documents/MicroRec/experiments/data/Results_Tags_with_Docker_S7_final_data.json')
df.head()

,Question Link,Question Title,Question Body,Accepted Answer Link,Accepted Answer Body,Question Tags,Accepted Answer Score,Github Links,Creation Date
0,390150,Authenticating against Active Directory with J...,<p>I have a simple task of authenticating agai...,394193,<p>Here's the code I put together based on exa...,<java><authentication><active-directory><ldap>,53,[{'link': 'https://github.com/jenkinsci/active...,2008-12-23T21:42:37.663
1,1197678,Using Thrift with Delphi Win32,<p>I'm interested in connecting to the Evernot...,1201828,<p><strong>Old Answer Replaced thanks to Leo:<...,<web-services><delphi><thrift><evernote>,4,"[{'link': 'https://github.com/apache/thrift', ...",2009-07-29T01:43:09.570
2,4918569,Connecting to Redis To Go with PHP,<p>I am a newbie with Redis...I just recently ...,4918702,"<p><a href=""https://github.com/nrk/predis"" rel...",<php><authentication><redis>,2,"[{'link': 'https://github.com/nrk/predis', 'id...",2011-02-07T06:19:19.033
3,5135145,node.js connect-auth app? example (user regist...,<p>I'm trying to get the hang of node.js and a...,5242315,"<p>Since i was mentioned by @jpstrikesback, i'...",<javascript><node.js><authentication><session>...,8,[{'link': 'https://github.com/kcbanner/connect...,2011-02-27T18:52:43.780
4,5475858,Yesod Persistent Examples,"<p>I'm planing around with Yesod, and I am hav...",5482134,<p>the source for the haskellers.com website i...,<web-services><sqlite><haskell><yesod>,4,[{'link': 'https://github.com/snoyberg/haskell...,2011-03-29T16:34:37.263


In [6]:
print(df['Github Links'].iloc[0])


[{'link': 'https://github.com/jenkinsci/active-directory-plugin', 'id': '394193', 'isDocker': True, 'docker_related': 'Dockerfile from src/test/resources/fixture/Dockerfile:\nFROM docker:dind\n# Keys were rotated in the edge repos, so we need to do this now\nRUN apk add -X https://dl-cdn.alpinelinux.org/alpine/v3.16/main -u alpine-keys --allow-untrusted\n# Workaround for  https://github.com/AdoptOpenJDK/openjdk-docker/issues/75\nRUN apk add --no-cache fontconfig ttf-dejavu openjdk11 bash tini bind-tools\nRUN apk add aufs-util --repository https://dl-cdn.alpinelinux.org/alpine/edge/testing\nRUN apk add maven --repository https://dl-cdn.alpinelinux.org/alpine/edge/community\n# Workaround for  https://github.com/AdoptOpenJDK/openjdk-docker/issues/75\nRUN ln -s /usr/lib/libfontconfig.so.1 /usr/lib/libfontconfig.so && \\\n    ln -s /lib/libuuid.so.1 /usr/lib/libuuid.so.1 && \\\n    ln -s /lib/libc.musl-x86_64.so.1 /usr/lib/libc.musl-x86_64.so.1\nENV LD_LIBRARY_PATH /usr/lib\n\nENV JAVA_HOME

In [7]:
df['label'] = df['Github Links'].apply(lambda x: x[0]['label'] if 'label' in x[0] else 'No Label')
df.head()


,Question Link,Question Title,Question Body,Accepted Answer Link,Accepted Answer Body,Question Tags,Accepted Answer Score,Github Links,Creation Date,label
0,390150,Authenticating against Active Directory with J...,<p>I have a simple task of authenticating agai...,394193,<p>Here's the code I put together based on exa...,<java><authentication><active-directory><ldap>,53,[{'link': 'https://github.com/jenkinsci/active...,2008-12-23T21:42:37.663,1317
1,1197678,Using Thrift with Delphi Win32,<p>I'm interested in connecting to the Evernot...,1201828,<p><strong>Old Answer Replaced thanks to Leo:<...,<web-services><delphi><thrift><evernote>,4,"[{'link': 'https://github.com/apache/thrift', ...",2009-07-29T01:43:09.570,2502
2,4918569,Connecting to Redis To Go with PHP,<p>I am a newbie with Redis...I just recently ...,4918702,"<p><a href=""https://github.com/nrk/predis"" rel...",<php><authentication><redis>,2,"[{'link': 'https://github.com/nrk/predis', 'id...",2011-02-07T06:19:19.033,2405
3,5135145,node.js connect-auth app? example (user regist...,<p>I'm trying to get the hang of node.js and a...,5242315,"<p>Since i was mentioned by @jpstrikesback, i'...",<javascript><node.js><authentication><session>...,8,[{'link': 'https://github.com/kcbanner/connect...,2011-02-27T18:52:43.780,2446
4,5475858,Yesod Persistent Examples,"<p>I'm planing around with Yesod, and I am hav...",5482134,<p>the source for the haskellers.com website i...,<web-services><sqlite><haskell><yesod>,4,[{'link': 'https://github.com/snoyberg/haskell...,2011-03-29T16:34:37.263,1774


In [8]:
def generate_random_sampling_min_m_max_n(df, min_answer=None, min_positive=None, min_negative=None, reset_index=True):
    np.random.seed(42)
    passage_dict = {}
    corpus_dict = {}
    answers_dict = {}
    api_list=[]
    
    gps = df.groupby('answer')

    for k, v in gps:
        if len(v) >= min_answer:
            api_list.append(k)
    df = df[df['answer'].isin(api_list)]
  
    
    gps = df.groupby('answer')
    
    title_list = df['title'].to_list()
    answer_list = df['answer'].to_list()
    indices = list(df.index)
    
   
    
    for idx, t in enumerate(title_list):
        corpus_dict[indices[idx]] = t
        
    for idx, t in enumerate(answer_list):
        answers_dict[indices[idx]] = t
        
    counter = 0
    
    for idx in indices:
        counter += 1
        if counter %10000 == 0: print(counter)
        label = answers_dict[idx]
        
        gp = gps.get_group(label)
        
        same_api_idx_gp = list(gp.index)
        diff_api_idx_gp = list(set(indices) - set(same_api_idx_gp))

        if len(same_api_idx_gp) > min_positive:
            same_api_idx_gp = np.random.choice(same_api_idx_gp, min_positive, replace=False).tolist()
            
        diff_api_idx_gp = np.random.choice(diff_api_idx_gp, min_negative, replace=False).tolist()

        passage_dict[idx] = [same_api_idx_gp, diff_api_idx_gp]
        

    return df, corpus_dict, answers_dict, passage_dict

def get_rel_doc(df, queries, Answers_dict):
    rel_dict ={}
    gps = df.groupby('answer')
    
    for q in queries:
        label = Answers_dict[q]
        
        gp = gps.get_group(label)
        
        same_api_idx_gp = list(gp.index)
        same_api_idx_gp = list(set(same_api_idx_gp)-set([q]))

        rel_dict[q] = [same_api_idx_gp]
        
    return rel_dict

def get_triplets(passage_dict):
    triplets= []
    for k, v in passage_dict.items():
        for x in v[0]:
            for y in v[1]:
                triplets.append([k, x, y])

    return triplets

In [9]:

df['answer'] = df['Github Links'].apply(lambda x: x[0]['label'] if 'label' in x[0] else 'No Label')
df['title'] = df['Question Title']
df.head(2)


,Question Link,Question Title,Question Body,Accepted Answer Link,Accepted Answer Body,Question Tags,Accepted Answer Score,Github Links,Creation Date,label,answer,title
0,390150,Authenticating against Active Directory with J...,<p>I have a simple task of authenticating agai...,394193,<p>Here's the code I put together based on exa...,<java><authentication><active-directory><ldap>,53,[{'link': 'https://github.com/jenkinsci/active...,2008-12-23T21:42:37.663,1317,1317,Authenticating against Active Directory with J...
1,1197678,Using Thrift with Delphi Win32,<p>I'm interested in connecting to the Evernot...,1201828,<p><strong>Old Answer Replaced thanks to Leo:<...,<web-services><delphi><thrift><evernote>,4,"[{'link': 'https://github.com/apache/thrift', ...",2009-07-29T01:43:09.570,2502,2502,Using Thrift with Delphi Win32


In [10]:
# filter dataset
df_filtered, corpus_dict, answers_dict, passage_dict = generate_random_sampling_min_m_max_n(
    df, min_answer=5, min_positive=10, min_negative=10
)
print(len(df_filtered))
df_filtered.head(2)

5689


,Question Link,Question Title,Question Body,Accepted Answer Link,Accepted Answer Body,Question Tags,Accepted Answer Score,Github Links,Creation Date,label,answer,title
5,5865086,How do you implement authentication in service...,<p>I'm investigating servicestack.net - but it...,5869158,"<p>Edit: \nThere is now a new <a href=""https:/...",<c#><security><authentication><servicestack>,25,[{'link': 'https://github.com/ServiceStack/Ser...,2011-05-03T04:40:23.070,2280,2280,How do you implement authentication in service...
8,7637356,HTTP Basic Custom Error Possible in Rails?,<p>I am building an application in Rails 2.3.1...,7640418,"<p>If you look at the <a href=""https://github....",<ruby-on-rails><ruby><authentication><basic-au...,5,"[{'link': 'https://github.com/rails/rails', 'i...",2011-10-03T15:48:00.580,2521,2521,HTTP Basic Custom Error Possible in Rails?


In [11]:
github_links = df_filtered['answer'].unique()
train_links, tmp_links = train_test_split(github_links, test_size=70, random_state=42, shuffle=True)
evaluate_links, test_links = tmp_links[:35], tmp_links[35:]

train_queries = []
for link in train_links:
    df_link = df_filtered[df_filtered['answer'] == link]
    train_queries += list(df_link.index)

evaluate_queries = []
for link in evaluate_links:
    df_link = df_filtered[df_filtered['answer'] == link]
    evaluate_queries += list(df_link.index)

test_queries = []
for link in test_links:
    df_link = df_filtered[df_filtered['answer'] == link]
    test_queries += list(df_link.index)

In [12]:
df_train = df_filtered.loc[train_queries]
df_tmp, train_corpus, train_answer, train_passage = generate_random_sampling_min_m_max_n(
    df_train, min_answer=5, min_positive=10, min_negative=10, reset_index=False
)

df_val = df_filtered.loc[evaluate_queries]
df_tmp, val_corpus, val_answer, val_passage = generate_random_sampling_min_m_max_n(
    df_val, min_answer=5, min_positive=10, min_negative=10, reset_index=False
)

In [13]:
train_rel_doc = get_rel_doc(df_filtered, train_queries, answers_dict)
evaluate_rel_doc = get_rel_doc(df_filtered, evaluate_queries, answers_dict)
test_rel_doc = get_rel_doc(df_filtered, test_queries, answers_dict)

# Create a unique folder for each iteration
data_folder = f'data/generated3/'
os.makedirs(data_folder, exist_ok=True)

df_filtered['index'] = df_filtered.index
df_filtered.to_json('data/generated3/dataset.json', orient='records')

with open(f'{data_folder}/train_corpus.json', 'w') as jsonfile:
    json.dump(train_corpus, jsonfile)

with open(f'{data_folder}/train_passage.json', 'w') as jsonfile:
    json.dump(train_passage, jsonfile)

with open(f'{data_folder}/val_corpus.json', 'w') as jsonfile:
    json.dump(val_corpus, jsonfile)

with open(f'{data_folder}/val_passage.json', 'w') as jsonfile:
    json.dump(val_passage, jsonfile)

# Save the data in separate JSON files for each split
with open(f'{data_folder}/Corpus_dict.json', 'w') as jsonfile:
     json.dump(corpus_dict, jsonfile)

with open(f'{data_folder}/Answers_dict.json', 'w') as jsonfile:
    json.dump(answers_dict, jsonfile)

with open(f'{data_folder}/Passage_dict.json', 'w') as jsonfile:
     json.dump(passage_dict, jsonfile)

with open(f'{data_folder}/train_queries.json', 'w') as jsonfile:
     json.dump(train_queries, jsonfile)

with open(f'{data_folder}/train_rel_doc.json', 'w') as jsonfile:
     json.dump(train_rel_doc, jsonfile)
    
with open(f'{data_folder}/evaluate_queries.json', 'w') as jsonfile:
    json.dump(evaluate_queries, jsonfile)

with open(f'{data_folder}/evaluate_rel_doc.json', 'w') as jsonfile:
     json.dump(evaluate_rel_doc, jsonfile)

with open(f'{data_folder}/test_queries.json', 'w') as jsonfile:
    json.dump(test_queries, jsonfile)

with open(f'{data_folder}/test_rel_doc.json', 'w') as jsonfile:
    json.dump(test_rel_doc, jsonfile)

/tmp/ipykernel_1482172/2258442150.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['index'] = df_filtered.index


### Verify passage dict and get_triplets are correct

In [14]:
triplets = get_triplets(val_passage)
for t in triplets[100]:
    row = df_filtered.loc[t]
    print(f'{row["Question Title"]}')
    print(f'{row["Github Links"][0]["link"]}')
    print()

PHP mail not being received, appears to be successful -- hosted on AWS
https://github.com/PHPMailer/PHPMailer

how to send mail using server-side language?
https://github.com/PHPMailer/PHPMailer

Kubernetes - Job scheduling API
https://github.com/kubernetes-client/java



### Verify *_rel_doc is correct

In [15]:
for doc, pairs in evaluate_rel_doc.items():
    row = df_filtered.loc[doc]

    positive = np.random.choice(pairs[0])
    positive = df_filtered.loc[positive]

    print(row['Question Title'], row['Github Links'][0]['link'])
    print(positive['Question Title'], positive['Github Links'][0]['link'])
    break

GMail SMTP avoid using direct password in PHP code https://github.com/PHPMailer/PHPMailer
How to send the email in PHP using Amazon SES https://github.com/PHPMailer/PHPMailer


### Verify no data leak between train / validation / test splits

In [16]:
train_q = []
for q in train_queries:
    train_q.append(corpus_dict[q])

val_q = []
for q in evaluate_queries:
    val_q.append(corpus_dict[q])

test_q = []
for q in test_queries:
    test_q.append(corpus_dict[q])

train_vs_val = set(train_q).isdisjoint(val_q)
train_vs_test = set(train_q).isdisjoint(test_q)
val_vs_test = set(val_q).isdisjoint(test_q)

print(train_vs_val, train_vs_test, val_vs_test)

True True True
